In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
from pytorch3d.loss import chamfer_distance

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
from pytorch3d.loss import chamfer_distance

sys.path.append(str(Path.cwd().parent))

from Helpers.data import PointCloudDataset
import Helpers.PointCloudOpen3d as pc

if torch.cuda.is_available():
    device = "cuda"

elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')

Using: cuda


In [23]:
# point_size = 1024
# train_dataset = PointCloudDataset("..Data/ModelNet40", point_size, 'train')
# train_loader = DataLoader(train_dataset, batch_size = 256, shuffle = True)
# print(len(train_dataset))

train_datasettt = PointCloudDataset("../Data/ModelNet40", point_size, 'train')
print(len(train_datasettt))

KeyboardInterrupt: 

In [4]:
x = next(iter(train_loader))['points'][0]
cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)

In [ ]:
feat = x

conv1 = nn.Conv2d(10)

torch.Size([1024, 3])

In [20]:
feat = x.T
conv1 = nn.Conv1d(3, 64, 1) # 64 kernels of size (3,1). Each kernel turns 3 points into 1 point 
conv2 = nn.Conv1d(64, 128, 1) # 128 kernels of size (64,1). Still operating on a per point level 
out = conv1(feat)
out = conv2(out)
out.shape

torch.Size([128, 1024])

In [ ]:
class ConvPointCloudAE(nn.Module):
    def __init__(self, config):
        pass 

    
    def encoder(self,x):
        pass 


    def decoder(self,x):
        pass


    def forward(self,x):
        latent_rep = self.encoder(x)
        reconstructed = self.decoder(x)
        return reconstructed

In [41]:
class PointCloudAE(nn.Module):
    def __init__(self, point_size, latent_size):
        super(PointCloudAE, self).__init__()
        
        self.latent_size = latent_size
        self.point_size = point_size
        
        self.conv1 = torch.nn.Conv1d(3, 1024, 1)
        self.conv2 = torch.nn.Conv1d(1024, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, self.latent_size, 1)
        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(self.latent_size)
        
        self.dec1 = nn.Linear(self.latent_size,1024)
        self.dec2 = nn.Linear(1024,2048)
        self.dec3 = nn.Linear(2048,self.point_size*3)

    def encoder(self, x): 
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, self.latent_size)
        return x
    
    def decoder(self, x):
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = self.dec3(x)
        return x.view(-1, self.point_size, 3)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    


net = PointCloudAE(point_size, 768)


device = torch.device("cuda:0")


net = net.to(device)

from pytorch3d.loss import chamfer_distance # chamfer distance for calculating point cloud distance

optimizer = optim.Adam(net.parameters(), lr=0.0005)

In [44]:

optimizer = optim.Adam(net.parameters(), lr=0.0001)

def train_epoch():
    epoch_loss = 0
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        data = data['points'].to(device)
        output = net(data.permute(0,2,1)) # transpose data for NumberxChannelxSize format
        loss, _ = chamfer_distance(data, output) 
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    if i == 0:
        return epoch_loss / 1
    return epoch_loss/i


train_loss_list = []  
test_loss_list = []  

for i in range(50000) :

    startTime = time.time()
    
    train_loss = train_epoch() #train one epoch, get the average loss
    train_loss_list.append(train_loss)

    
    epoch_time = time.time() - startTime
    
    if i % 100 == 0:
        print(train_loss)

0.0049487215542602676
0.001842792481617179
0.0014271337138272303
0.001220517770148987
0.0010973226582241613
0.0010287557866662568


KeyboardInterrupt: 

In [45]:
torch.save(net.state_dict(), './50kepoch_model')

In [50]:
x = next(iter(train_loader))['points'][0]

cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)


with torch.no_grad():
    xx = x.T.unsqueeze(0).to(device)
    rec_x = np.array(net(xx)[0].to('cpu'))
    cloud = pc.get_point_cloud(rec_x)
    pc.visualize_point_cloud(cloud)

In [103]:

x = next(iter(train_loader))['points'][0]

# print(x[:,i].min())
x_min = x.min()
x_max = x.max()

x_norm = (x - x_min) / (x_max - x_min)

cloud = pc.get_point_cloud(x_norm)
pc.visualize_point_cloud(cloud)

print(x_norm)

tensor([[0.4817, 0.6104, 0.3859],
        [0.4709, 0.6067, 0.4072],
        [0.4908, 0.6125, 0.3900],
        ...,
        [0.5262, 0.5955, 0.4208],
        [0.5483, 0.8067, 0.4412],
        [0.5769, 0.8080, 0.4399]])


In [100]:
x_norm


tensor([[0.5825, 0.5014, 0.1379],
        [0.5915, 0.4976, 0.1370],
        [0.8215, 0.3296, 0.1172],
        ...,
        [0.6037, 0.3774, 0.0789],
        [0.6028, 0.3761, 0.0731],
        [0.6030, 0.3761, 0.0733]])